## README

### Random forest

#### 100% testing, 100% validation:

```
[('quantity', 0.0079976101188701763),
 ('voucherAmount', 0.013113168566589808),
 ('productGroup', 0.013521791424342219),
 ('paymentMethod', 0.023095387514901027),
 ('voucherID', 0.023673399468102159),
 ('deviceID', 0.02972387355282638),
 ('rrp', 0.034301681510350089),
 ('price', 0.050416333839450621),
 ('sizeCode', 0.081037284731151293),
 ('articleID', 0.099016018175452983),
 ('colorCode', 0.102658103196932),
 ('mmdd', 0.11712529498716262),
 ('orderDate', 0.11765229923719016),
 ('budget', 0.13307758983005333),
 ('total_order', 0.15359016384662513)]
```

error: **40909**

```
[[1106669   10130       4       2       0       0]
 [  30395 1173596      14       0       0       0]
 [     98     105    3546       0       0       0]
 [      5       5       1     181       0       0]
 [      1       2       0       0      32       0]
 [      3       1       0       0       0      24]]
```

##### + fitur farah

```
[('quantity', 0.0017582530808943407),
 ('gpriceDiff', 0.0048983941549550864),
 ('priceDiff', 0.013412936473421951),
 ('sizeCode2', 0.02654573612543476),
 ('price', 0.02953846862187191),
```

#### 70% testing, 30% validation:
```
[('voucherAmount', 0.008844496709481408),
 ('quantity', 0.0093324031702163406),
 ('productGroup', 0.014387262675345536),
 ('voucherID', 0.016277729810941808),
 ('deviceID', 0.019815761309476355),
 ('paymentMethod', 0.019978720674466634),
 ('months', 0.026730617852121656),
 ('rrp', 0.02948235001143798),
 ('choice_order', 0.035629408368906129),
 ('price', 0.0417470189282238),
 ('sizeCode', 0.044808807069813467),
 ('articleID', 0.05424272890421826),
 ('colorCode', 0.05976713876698625),
 ('order_order', 0.062850904570996804),
 ('mmdd', 0.070210862709374419),
 ('orderDate', 0.070721914304349986),
 ('after_voucher', 0.075593711311003509),
 ('budget', 0.081893112411723187),
 ('customerID', 0.084632659177736408),
 ('total_order', 0.084915523649828545),
 ('return_prob', 0.088136867613351599)]

Error:  259037

[[222443 112565     62      3      0      0]
 [144594 216398    190      3      1      0]
 [   400    551    158      0      0      0]
 [    35     13      4      5      0      0]
 [     4      3      2      0      0      0]
 [     3      3      0      2      0      2]]
 ```

In [10]:
label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from datetime import datetime
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from __future__ import division
%matplotlib inline
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

In [11]:
# Dari Farah:

def getSeason(month):
    if month <3 or month==12:
        return "Winter"
    elif month >=3 and month<6:
        return "Spring"
    elif month >=6 and month <9:
        return "Summer"
    else:
        return "Fall"
    
def changeSizeCode(size):
    if size in ['24','25','26','27','28','29','30','31','100']:
        return 'XL'
    elif size in ['32','33','34']:
        return 'XS'
    elif size in ['36','38','75']:
        return 'S'
    elif size in ['40','42','80']:
        return 'M'
    elif size in ['44','85','90','95']:
        return 'L'    
    
def grouppriceDiff(priceDiff):
    if priceDiff < 0:
        return 1
    elif priceDiff == 0:
        return 2
    elif priceDiff > 0 and priceDiff <= 9.99:
        return 3
    else:
        return 4

In [12]:
#df=pd.read_csv('orders_train.txt', sep=';'); df

In [13]:
#joblib.dump(df, 'df.pkl') 
orig_df=joblib.load('df.pkl')

In [14]:
df=orig_df.copy()

In [15]:
df['mmdd']=orig_df.orderDate.str[-5:]
df['months']=orig_df.orderDate.str[-5:-3].astype(np.int)
#df['sizeCode2']=df.sizeCode.apply(changeSizeCode)
#df['season']= df.months.apply(getSeason)
#df['priceDiff'] = df['rrp']-(df['price']/df['quantity'])
#df['gpriceDiff'] = df['priceDiff'].apply(grouppriceDiff)

In [16]:
# Berapa total harga dari suatu order

order=df[['orderID', 'price']].groupby('orderID').sum()
order.columns=['total_order']
df=df.set_index('orderID').join(order).reset_index()
del order

# budget rata-rata customer

budget=df[['customerID','total_order']].groupby('customerID').mean()
budget.columns=['budget']
df=df.set_index('customerID').join(budget).reset_index()
del budget

# Return probability

df_return_probability=df[['articleID','returnQuantity','quantity']].groupby('articleID').sum()
df_return_probability['return_probability']=df_return_probability.returnQuantity/df_return_probability.quantity
return_prob_dict=df_return_probability['return_probability'].to_dict()
del df_return_probability
df['return_prob']= df.articleID.apply(lambda x: return_prob_dict.get(x,0.5))

# total_order - voucherAmount
df['after_voucher']=df.total_order - df.voucherAmount

# urutan
df['order_order'] = df[['customerID','orderID']].groupby(['customerID']).cumcount();
df['choice_order']= df[['orderID','articleID']].groupby(['orderID']).cumcount();

In [17]:
"""# wednesday
df['wednesday'] =orig_df.orderDate.apply(pd.to_datetime).apply(lambda x: x.weekday())
df.wednesday[df.wednesday!=2]=0
df.wednesday[df.wednesday==2]=1"""

"# wednesday\ndf['wednesday'] =orig_df.orderDate.apply(pd.to_datetime).apply(lambda x: x.weekday())\ndf.wednesday[df.wednesday!=2]=0\ndf.wednesday[df.wednesday==2]=1"

In [19]:
#joblib.dump(df, 'df-04-14.pkl') 
#orig_df=joblib.load('df-04-14.pkl')
#df.to_csv("df-04-14.csv")

In [11]:
#iseng ubah float ke integer
df.price=df.price.astype(np.int)
df.budget=df.budget.astype(np.int)

#Konversi kategori/object ke numerik
print "Konversi kategori/object ke numerik:"

#Cari kolom yang tipenya object, bukan integer maupun float
object_columns=df.loc[:, df.dtypes == object].columns

for col in object_columns:
    print col
    le=LabelEncoder()
    # Konversi deh
    df[col]=le.fit_transform(df[col])
    
#df['price_delta']=df.rrp-df.price;# df.price_delta.head()

Konversi kategori/object ke numerik:
customerID
orderID
orderDate
articleID
sizeCode
voucherID
paymentMethod
mmdd


In [93]:
df.head()

,customerID,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,...,paymentMethod,returnQuantity,mmdd,months,total_order,budget,return_prob,after_voucher,order_order,choice_order
0,0,1452,2,2943,3999,23,17,1,27,27.99,...,2,0,2,1,27.99,69,0.509636,27.99,0,0
1,0,4937,6,2948,1999,23,17,1,25,25.99,...,2,1,6,1,25.99,69,0.427653,25.99,1,0
2,0,12400,14,1177,1993,13,4,1,39,39.99,...,2,0,14,1,39.99,69,0.473249,39.99,2,0
3,0,13403,15,2418,1995,13,15,1,59,59.99,...,2,1,15,1,119.98,69,0.503149,119.98,3,0
4,0,13403,15,2418,1999,14,15,1,59,59.99,...,2,1,15,1,119.98,69,0.503149,119.98,4,1


In [20]:
# Selangkah sebelum ditrain
label='returnQuantity'

# Drop NaN
df2=df.sort("orderID").dropna()

Xdf=df2.drop([label,"orderID"], axis=1)
Xdf.rrp=Xdf.rrp.astype(np.int)

X=Xdf.values
y=df2.returnQuantity.values

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [110]:
rfc=RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [111]:
from itertools import izip as zip
pair=zip(Xdf.columns, rfc.feature_importances_)
t1=[i for i in pair]; t1
#
from operator import itemgetter
sorted(t1, key=itemgetter(1))

[('voucherAmount', 0.008844496709481408),
 ('quantity', 0.0093324031702163406),
 ('productGroup', 0.014387262675345536),
 ('voucherID', 0.016277729810941808),
 ('deviceID', 0.019815761309476355),
 ('paymentMethod', 0.019978720674466634),
 ('months', 0.026730617852121656),
 ('rrp', 0.02948235001143798),
 ('choice_order', 0.035629408368906129),
 ('price', 0.0417470189282238),
 ('sizeCode', 0.044808807069813467),
 ('articleID', 0.05424272890421826),
 ('colorCode', 0.05976713876698625),
 ('order_order', 0.062850904570996804),
 ('mmdd', 0.070210862709374419),
 ('orderDate', 0.070721914304349986),
 ('after_voucher', 0.075593711311003509),
 ('budget', 0.081893112411723187),
 ('customerID', 0.084632659177736408),
 ('total_order', 0.084915523649828545),
 ('return_prob', 0.088136867613351599)]

In [115]:
y_pred=rfc.predict(X_test)
print "Error: ", dmc_error(y_test, y_pred)
print confusion_matrix(y_test, y_pred)

Error:  259037
[[222443 112565     62      3      0      0]
 [144594 216398    190      3      1      0]
 [   400    551    158      0      0      0]
 [    35     13      4      5      0      0]
 [     4      3      2      0      0      0]
 [     3      3      0      2      0      2]]


In [113]:
"""from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_jobs=-1)
rfr.fit(X_train, y_train)
y_pred_rfr=rfc.predict(X_test)
y_pred_rfr_i=[int(label) for label in y_pred_rfr]
print dmc_error(y_test, y_pred_rfr_i)
print confusion_matrix(y_test, y_pred_rfr_i)
print y_pred_rfr_i"""

'from sklearn.ensemble import RandomForestRegressor\nrfr=RandomForestRegressor(n_jobs=-1)\nrfr.fit(X_train, y_train)\ny_pred_rfr=rfc.predict(X_test)\ny_pred_rfr_i=[int(label) for label in y_pred_rfr]\nprint dmc_error(y_test, y_pred_rfr_i)\nprint confusion_matrix(y_test, y_pred_rfr_i)\nprint y_pred_rfr_i'